<a href="https://akademie.datamics.com/kursliste/">![title](bg_datamics_top.png)</a>

<center><em>© Datamics</em></center><br><center><em>Besuche uns für mehr Informationen auf <a href='https://akademie.datamics.com/kursliste/'>www.akademie.datamics.com</a></em></center>

# Grundlegende Algorithmenmethoden

Testen wir unsere zuvor optimierte Technologieportfolio-Strategie alorithmisch mit Quantopian!

#### DIESER CODE FUNKTIONERT NUR AUF QUANTOPIAN. JEDE ZELLE HÄNGT MIT EINEM TEIL DES VIDEOKURSES ZUSAMMEN. SCHAUE DIR DAS VIDEO AN, UM DARÜBER MEHR ZU ERFAHREN!

**initialize()**

initialize() wird genau einmal aufgerufen, sobald unser Algorithmus beginnt und fordert "context" als Input.

"context" ist ein erweitertes Python Wörterbuch (dictionary), das für die Erhaltung des Status während unseres Rückvergleichs (backtest) oder Livehandels verwendet wird. Es kann in verschiedenen Teilen unseres Algorithmus zitiert werden. "context" sollte anstelle von globalen Variablen im Algorithmus verwendet werden. Auf die Eigenschaften kann mit der Punkt-Notation zugegriffen werden (context.eine_eigenschaft).

** handle_data() **

handle_data() wird einmal am Ende jeder Minute aufgerufen und fordert "context" und "data" als Input. "context" ist eine Referenz auf dasselbe Wörterbuch wie in initialize() und "data" ist ein Objekt, das verschiedene API-Funktionen enthält.

## Unser optimiertes Portfolio von Technologieaktien

Benutzen wir unser Portfolio von Technologieaktien, das wir zuvor berechnet haben. Denk dran, dass handle_data() unser Portfolio jede Minute anpasst! Das mag für manche Algorithmen unvernünftig sein, aber in diesem Beispiel werden wir einfach mit diesen grundlegenden Funktionen weitermachen.

In [1]:
def initialize(context):
    # Referenz auf Technologieaktien
    context.aapl = sid(24)
    context.csco = sid(1900)
    context.amzn = sid(16841)

def handle_data(context, data):
    # Positioniere unsere Portfolio-Optimierung!
    order_target_percent(context.aapl, .27)
    order_target_percent(context.csco, .20)
    order_target_percent(context.amzn, .53)

### Aktuelle Daten erfassen

### data.current()
data.current() kann verwendet werden, um die aktuellsten Werte eines oder mehreren gegebenen Felder für einen oder mehrere Posten abzufragen. data.current() erfordert zwei Argumente: Den Posten oder die Liste der Posten und das Feld oder die Liste der Felder, die abgefragt werden. Mögliche Felder sind z.B. 'price', 'open', 'high', 'low', 'close', und 'volume'. Der Output hängt von den Inputtypen ab.

In [3]:
def initialize(context):
    # Referenz auf Technologieaktien
    context.techies = [sid(16841),sid(24),sid(1900)]

def handle_data(context, data):
    # Positioniere unsere Portfolio-Optimierung!
    tech_close = data.current(context.techies,'close')
    print(type(tech_close)) # Pandas-Reihen
    print(tech_close) # Schlusspreise

##### Achtung! Du kannst data.is_stale(sid#)) verwenden, um zu überprüfen, ob die Ergebnisse von data.current() am aktuellen "bar" (Zeitfenster) generiert wurden oder im Voraus eingetragen wurden.

### Handelsmöglichkeiten ausfindig machen

### data.can_trade()

data.can_trade() wird verwendet, um zu bestimmen, ob ein oder mehrere Posten zur Zeit auf einer unterstützen Börse gelistet ist und bestellt werden kann. Falls data.can_trade() True für einen bestimmten Posten eines Minuten"bars" ist, können wir eine Bestellung für diesen Posten in dieser Minute in Auftrag geben. Dies ist ein wichtiger Schutz in unserem Algorithmus, falls wir die Wertpapiere, die wir handeln wollen, manuell aussuchen. Es erfordert nur ein Argument: einen Posten oder eine Liste von Posten.

In [5]:
def initialize(context):
    # Referenz auf Amazon
    context.amzn = sid(16841)
    
def handle_data(context, data):
    # Dies verhindert, dass wir eine Exception auslösen!
    if data.can_trade(sid(16841)):
        order_target_percent(context.amzn, 1.0)

# Historische Daten überprüfen

Wenn Dein Algorithmus data.history auf Marktwerte anwendet, werden die zurückgegebenen Daten an Splits, Fusionen und Dividenden des aktuellen Simulationsdatums angepasst. In anderen Worten: Wenn Dein Algorithmus ein historisches Zeitfenster der Preise anfragt, und es einen Split in der Mitte dieses Fensters gibt, wird der erste Teil des Fensters an den Split angepasst. Diese Anpassung wird so vorgenommen, dass dein Algorithmus aussagekräftige Berechnungen aufgrund der Werte im Fenster ausführen kann.

Dieser Code fragt die Preishistorie der letzten 20 Tage für einen statischen Satz von Wertpapieren ab. Genauer gesagt liefert dies den täglichen Schlusspreis der letzten 20 Tage, inklusive dem Preis des aktuellen Tages. Marktwertpreise sind Split- und Dividenden-angepasst für das aktuelle Datum der Simulation:

In [8]:

def initialize(context):
    # AAPL, MSFT und SPY
    context.assets = [sid(24), sid(1900), sid(16841)]

def before_trading_start(context,data):
    price_history = data.history(context.assets,fields="price", bar_count=5, frequency="1d")
    
    print(price_history)


Das Feld bar_count gibt die Anzahl der Tage oder Minuten an, die in dem von der history-Funktion zurückgegebenen pandas Datenframe betrachtet werden soll. Dieser Parameter akzeptiert nur Integerwerte.

Das Feld frequency gibt an, wie oft die Stichproben genommen werden: täglich oder minütlich. Mögliche Inputs sind '1d' oder '1m'. Für andere Frequenzen musst Du pandas Funktion resample verwenden.

### Beispiele
Unten sind Beispiele für Code, zusammen mit Erklärungen der zurückgegebenen Daten.

### Tägliche Historie

Verwende '1d' als Frequenz. Das zurückgegebene Datenframe liegt immer in täglichen Balken vor. Die Balken umfassen nie mehr als einen Handelstag. Für US Vermögenswerte erfasst ein täglicher Balken die Handelsaktivität während der Marktstunden (üblicherweise 9:30 - 16:00 Uhr ET). Für US Termingeschäfte umfasst ein täglicher Balken die Handelsaktivität von 18:00 bis 18:00 Uhr ET (24 Stunden). Zum Beispiel umfasst der tägliche Balken von Montag die Handelsaktivität von Sonntag 18:00 Uhr bis 18:00 am Montag. Der tägliche Balken von Dienstag geht von Montag 18:00 bis Dienstag 18:00 usw. Für jede Postengruppe wird der letzte Balken, falls er nur teilweise vorhanden ist, aus den Minuten des aktuellen Tages aufgebaut.

### Beispiele (angenommen context.assets existiert):

* data.history(context.assets, "price", 1, "1d") gibt den aktuellen Preis zurück
* data.history(context.assets, "volume", 1, "1d") gibt das Volumen seit Öffnung des aktuellen Tages zurück, auch wenn unvollständig.
* data.history(context.assets, "price", 2, "1d") gibt den gestrigen Schlusspreis und den aktuellen Preis zurück.
* data.history(context.assets, "price", 6, "1d") gibt die Preise der letzten 5 Tage und den aktuellen Preis zurück.


### Minütliche Historie

Verwende '1m' als Frequenz.

### Beipiele (angenommen context.assets existiert):

* data.history(context.assets, "price", 1, "1m") gibt den aktuellen Preis zurück
* data.history(context.assets, "price", 2, "1m") gibt den Schlusspreis der vergangenen Minute und den aktuellen Preis zurück.
* data.history(context.assets, "volume", 60, "1m") gibt das Volumen der letzten 60 Minuten zurück.

# Zeitplanung (Scheduling)

Verwende schedule_function, um anzuzeigen, wann Du andere Funktionen ausführen lassen willst. Die übergebenen Funktionen müssen "context" und "data" als Parameter besitzen.

In [2]:
def initialize(context):
    context.appl = sid(49051)

    # Am Anfang der Handelswoche
    # Bei Öffnung des Marktes sollen 10% des Portfolios Apple-Aktien sein
    schedule_function(open_positions, date_rules.week_start(), time_rules.market_open())
    
    # Am Ende der Handelswoche
    # 30 Minuten vor Marktschluss alle Apple-Aktien loswerden. 
    schedule_function(close_positions, date_rules.week_end(), time_rules.market_close(minutes=30))

def open_positions(context, data):
    order_target_percent(context.appl, 0.10)

def close_positions(context, data):
    order_target_percent(context.appl, 0)

# Portfolio Informationen

Du kannst Informationen zu Deinem Portfolio bekommen und aufzeichnen!

In [3]:
def initialize(context):
    context.amzn = sid(16841)
    context.ibm = sid(3766)

    schedule_function(rebalance, date_rules.every_day(), time_rules.market_open())
    schedule_function(record_vars, date_rules.every_day(), time_rules.market_close())

def rebalance(context, data):
    # Die Hälfte unseres Portfolios "long" auf Amazon
    order_target_percent(context.amzn, 0.50)
    # Die Hälfte "short" auf IBM
    order_target_percent(context.ibm, -0.50)

def record_vars(context, data):

    # Die Auszählung graphisch darstellen
    record(amzn_close=data.current(context.amzn,'close'))
    record(ibm_close=data.current(context.ibm,'close'))

# Verzögerung (Slippage) und Kommission (Commission)

### Verzögerung (Slippage)
Verzögerung ist, wenn eine Simulation den Einfluss von Bestellungen auf die Erfüllrate (fillrate) und den Ausführungspreis (execution price), den sie erhalten, schätzt. Wenn eine Bestellung für einen Handel in Auftrag gegeben wird, wird der Markt beeinflusst. Kaufaufträge treiben Preise hoch, Verkaufaufträge runter; dies wird grundsätzlich als Preiseinfluss (price_impact) eines Handels bezeichnet. Zusätzlich werden Handelsbestellungen nicht unbedingt sofort erfüllt. Erfüllraten hängen von der Größe der Bestellung und dem aktuellen Handelsvolumen des bestellten Wertpapiert ab. Das Volumenlimit (volume_limit) bestimmt den Anteil des Handelsvolumens eines Wertpapier, der von Deinem Algorithmus benutzt werden kann.

In Rückvergleichen und nicht-Makler-Papierhandel (Quantopian Papierhandel) kann ein Verzögerungsmodell in initialize() mithilfe von set_slippage() verwendet werden. Es gibt verschiedene eingebaute Verzögerungsmodelle, die benutzt werden können, sowie die Option, ein benutzerdefiniertes Modell einzurichten. Im Defaultmodus (ohne ein gewähltes Verzögerungsmodell) wird das folgende Volume Share Verzögerungsmodell verwendet:

In [ ]:
set_slippage(slippage.VolumeShareSlippage(volume_limit=0.025, price_impact=0.1))

Mit dem Defaultmodell: wenn eine Bestellung für 60 Aktien für einen bestimmten Bestand in Auftrag gegeben wird, werden 1000 Aktien dieses Bestands in jeder der nächsten paar Minuten gehandelt und das Volumenlimit ist 0.025, denn unser Handelsauftrag wird in drei Aufträge aufgeteilt (25, 25 und 10 Aktien), die über die nächsten drei Minuten verteilt ausgeführt werden.

Am Ende jedes Tages werden alle offenen Aufträge annulliert, also ist das Handeln von flüssigen Beständen generell eine gute Idee. Zusätzlich werden Aufträge, die genau zu Marktschluss getätigt werden, keine Zeit haben, um ausgeführt zu werden und annuliert werden.

### Kommission (Commission)

Um die Kosten eines Handels zu spezifzieren können wir ein Kommissionsmodell in initialize() mithilfe von set_commission() angeben. Im Defaultmodus (wenn kein Kommissionsmodell gewählt wird), wird das folgende Kommissionsmodell verwendet:

In [ ]:
set_commission(commission.PerShare(cost=0.0075, min_trade_cost=1))

Das Default-Kommissionsmodell kostet 0.0075 Dollar pro Aktie, mit minimalen Handelskosten von 1 Dollar.

Verzögerungs- und Kommisionsmodelle können einen Einfluss auf die Leistung in einem Rückvergleich haben. Die Defaultmodelle von Quantopian sind ziemlich realistisch und es wird dringend empfohlen, diese zu verwenden.

# Super gemacht!

Das waren alle Grundlagen des Quantopian-Tutorials! Mit diesen wichtigsten Funktionen weißt du eigentlich schon genug, um mit dem Trading zu beginnen!